In [ ]:
import pygame
import random
import sys
import time
from pygame.locals import *

# Initialize pygame
pygame.init()

# Game settings
SCREEN_WIDTH = 400
SCREEN_HEIGHT = 600
FPS = 60
SPEED = 5
SCORE = 0
COINS_COLLECTED = 0
COIN_SPEED_BOOST = 5  # Number of coins needed to increase speed

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)

# Set up display
DISPLAYSURF = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Car Game")
FramePerSec = pygame.time.Clock()

# Load images
try:
    coin_img = pygame.image.load('coin.png')
    enemy_img = pygame.image.load('enemy.png')
    player_img = pygame.image.load('player.png')
    road_img = pygame.image.load('road.jpg')
    
    # Scale images
    coin = pygame.transform.scale(coin_img, (30, 30))  # Bigger coin size
    enemy = pygame.transform.scale(enemy_img, (150, 60))
    player = pygame.transform.scale(player_img, (150, 60))
    background = pygame.transform.scale(road_img, (SCREEN_WIDTH, SCREEN_HEIGHT))
except pygame.error as e:
    print(f"Error loading images: {e}")
    sys.exit()

# Fonts
font = pygame.font.SysFont("Verdana", 60)
font_small = pygame.font.SysFont("Verdana", 20)
game_over_text = font.render("Game Over", True, BLACK)

class Enemy(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = enemy
        self.rect = self.image.get_rect()
        self.rect.center = (random.randint(40, SCREEN_WIDTH-40), 0)
    
    def move(self):
        global SCORE
        self.rect.move_ip(0, SPEED)
        if self.rect.top > SCREEN_HEIGHT:
            SCORE += 1
            self.rect.top = 0
            self.rect.center = (random.randint(40, SCREEN_WIDTH-40), 0)

class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = player
        self.rect = self.image.get_rect()
        self.rect.center = (160, 520)
    
    def move(self):
        pressed_keys = pygame.key.get_pressed()
        if self.rect.left > 0 and pressed_keys[K_LEFT]:
            self.rect.move_ip(-5, 0)
        if self.rect.right < SCREEN_WIDTH and pressed_keys[K_RIGHT]:
            self.rect.move_ip(5, 0)
        if self.rect.top > 0 and pressed_keys[K_UP]:
            self.rect.move_ip(0, -5)
        if self.rect.bottom < SCREEN_HEIGHT and pressed_keys[K_DOWN]:
            self.rect.move_ip(0, 5)

class Coin(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = coin  # Using the bigger coin image
        self.value = 1  # All coins worth 1 point
        self.rect = self.image.get_rect()
        self.reset_position()
    
    def reset_position(self):
        self.rect.center = (random.randint(40, SCREEN_WIDTH-40), 
                          random.randint(40, SCREEN_HEIGHT-40))

# Initialize sprites
P1 = Player()
E1 = Enemy()
C1 = Coin()

# Create sprite groups
enemies = pygame.sprite.Group(E1)
coins = pygame.sprite.Group(C1)
all_sprites = pygame.sprite.Group(P1, E1, C1)

# Speed increase event
INC_SPEED = pygame.USEREVENT + 1
pygame.time.set_timer(INC_SPEED, 1000)

while True:
    for event in pygame.event.get():
        if event.type == INC_SPEED:
            SPEED += 0.2
        if event.type == QUIT:
            pygame.quit()
            sys.exit()
    
    DISPLAYSURF.blit(background, (0, 0))
    
    score_text = font_small.render(f'Score: {SCORE}', True, BLACK)
    coins_text = font_small.render(f'Coins: {COINS_COLLECTED}', True, BLACK)
    DISPLAYSURF.blit(score_text, (10, 10))
    DISPLAYSURF.blit(coins_text, (SCREEN_WIDTH-110, 10))
    
    for entity in all_sprites:
        DISPLAYSURF.blit(entity.image, entity.rect)
        if hasattr(entity, 'move'):
            entity.move()
    
    if pygame.sprite.spritecollideany(P1, coins):
        COINS_COLLECTED += C1.value
        C1.reset_position()
        
        if COINS_COLLECTED % COIN_SPEED_BOOST == 0:
            SPEED += 0.5
    
    if pygame.sprite.spritecollideany(P1, enemies):
        try:
            crash_sound = pygame.mixer.Sound('car.mp3')
            crash_sound.play()
        except:
            pass
        
        time.sleep(1)
        DISPLAYSURF.fill(RED)
        DISPLAYSURF.blit(game_over_text, (30, 250))
        pygame.display.update()
        time.sleep(2)
        pygame.quit()
        sys.exit()
    
    pygame.display.update()
    FramePerSec.tick(FPS)

SystemExit: 

In [4]:
import pygame
import random
import time

pygame.init()

WIDTH, HEIGHT = 600, 400
GRID = 20
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
BLACK = (0, 0, 0)

screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Snake Game")
clock = pygame.time.Clock()

snake = [(100, 100), (90, 100), (80, 100)]
direction = (GRID, 0)

score = 0
level = 1
speed = 10
game_over = False

class Food:
    def __init__(self):
        self.x = 0
        self.y = 0
        self.color = RED
        self.points = 1
        self.spawn_time = 0
        self.life = 0
    
    def new(self):
        while True:
            self.x = random.randint(0, (WIDTH//GRID)-1)*GRID
            self.y = random.randint(0, (HEIGHT//GRID)-1)*GRID
            if (self.x, self.y) not in snake:
                break
        
        if random.random() < 0.2:
            self.color = BLUE
            self.points = 3
            self.life = 5
        else:
            self.color = RED
            self.points = 1
            self.life = 0
        
        self.spawn_time = time.time()
    
    def check_time(self):
        if self.life > 0 and (time.time()-self.spawn_time) > self.life:
            return True
        return False

food = Food()
food.new()

running = True
while running:
    screen.fill(WHITE)
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_UP and direction != (0, GRID):
                direction = (0, -GRID)
            elif event.key == pygame.K_DOWN and direction != (0, -GRID):
                direction = (0, GRID)
            elif event.key == pygame.K_LEFT and direction != (GRID, 0):
                direction = (-GRID, 0)
            elif event.key == pygame.K_RIGHT and direction != (-GRID, 0):
                direction = (GRID, 0)
    
    if not game_over:
        new_x = snake[0][0] + direction[0]
        new_y = snake[0][1] + direction[1]
        new_head = (new_x, new_y)
        
        if new_x < 0 or new_x >= WIDTH or new_y < 0 or new_y >= HEIGHT or new_head in snake:
            game_over = True
        
        snake.insert(0, new_head)
        
        if new_head == (food.x, food.y):
            score += food.points
            if score % 5 == 0:
                level += 1
                speed += 2
            food.new()
        else:
            snake.pop()
        
        if food.check_time():
            food.new()
    
    pygame.draw.rect(screen, food.color, (food.x, food.y, GRID, GRID))
    
    for part in snake:
        pygame.draw.rect(screen, GREEN, (part[0], part[1], GRID, GRID))
    
    font = pygame.font.Font(None, 24)
    text = font.render(f"Score: {score} Level: {level}", True, BLACK)
    screen.blit(text, (10, 10))
    
    if game_over:
        big_font = pygame.font.Font(None, 48)
        over_text = big_font.render("GAME OVER!", True, BLACK)
        screen.blit(over_text, (WIDTH//2-100, HEIGHT//2-24))
    
    pygame.display.flip()
    clock.tick(speed)

pygame.quit()

In [8]:
import pygame
import math


pygame.init()

fps = 60  
timer = pygame.time.Clock()  
WIDTH = 800  
HEIGHT = 600

active_shape = 0  
active_color = 'white'  

screen = pygame.display.set_mode([WIDTH, HEIGHT])
pygame.display.set_caption("Paint Program")  

drawings = []

def create_menu(current_color):
    
    circle = pygame.draw.rect(screen, 'white', [10, 10, 50, 50])
    pygame.draw.circle(screen, 'black', (35, 35), 20, 2)  
    
    rect = pygame.draw.rect(screen, 'white', [70, 10, 50, 50])
    pygame.draw.rect(screen, 'black', [80, 20, 30, 25], 2) 
    
    square = pygame.draw.rect(screen, 'white', [130, 10, 50, 50])
    pygame.draw.rect(screen, 'black', [140, 20, 25, 25], 2)
    
    right_tri = pygame.draw.rect(screen, 'white', [190, 10, 50, 50])
    pygame.draw.polygon(screen, 'black', [(200, 40), (230, 40), (200, 20)], 2)
    
    equi_tri = pygame.draw.rect(screen, 'white', [250, 10, 50, 50])
    pygame.draw.polygon(screen, 'black', [(255, 40), (275, 40), (265, 20)], 2)
    
    rhombus = pygame.draw.rect(screen, 'white', [310, 10, 50, 50])
    pygame.draw.polygon(screen, 'black', [(320, 25), (335, 40), (350, 25), (335, 10)], 2)
    
    brushes = [
        [circle, 0],  
        [rect, 1],     
        [square, 2],    
        [right_tri, 3],  
        [equi_tri, 4],  
        [rhombus, 5]    
    ]
    
    pygame.draw.circle(screen, current_color, (500, 35), 20)
    pygame.draw.circle(screen, 'black', (500, 35), 20, 2)  
   
    erase = pygame.image.load('eraser.png')
    eraser=pygame.transform.scale(erase,(25,25))
    eraser_rect = eraser.get_rect(topleft=(WIDTH - 200, 7))
    eraser_rect.width = eraser_rect.height = 25
    screen.blit(eraser, [WIDTH - 200, 7, 25, 25])
    blue = pygame.draw.rect(screen, (0, 0, 255), [WIDTH - 80, 10, 25, 25])    
    red = pygame.draw.rect(screen, (255, 0, 0), [WIDTH - 95, 10, 25, 25])     
    green = pygame.draw.rect(screen, (0, 255, 0), [WIDTH - 110, 10, 25, 25])   
    yellow = pygame.draw.rect(screen, (255, 255, 0), [WIDTH - 125, 10, 25, 25]) 
    pink = pygame.draw.rect(screen, (255, 193, 202), [WIDTH - 140, 10, 25, 25])   
    purple = pygame.draw.rect(screen, (255, 0, 255), [WIDTH - 155, 10, 25, 25]) 
    black = pygame.draw.rect(screen, (0, 0, 0), [WIDTH - 170, 10, 25, 25])     
    
    color_buttons = [blue, red, green, yellow, pink, purple, black]
    color_values = [
        (0, 0, 255),    
        (255, 0, 0),     
        (0, 255, 0),     
        (255, 255, 0),   
        (255, 193, 202), 
        (255, 0, 255),   
        (0, 0, 0),       
         
    ]
    
    return brushes, color_buttons, color_values

def draw_all_shapes(all_drawings):
   
    for color, position, shape_type in all_drawings:
        if color == (255, 255, 255): 
            pygame.draw.rect(screen, color, [position[0] - 15, position[1] - 15, 30, 30])
        else:
            if shape_type == 0:  
                pygame.draw.circle(screen, color, position, 15, 2)
            elif shape_type == 1:  
                pygame.draw.rect(screen, color, [position[0] - 20, position[1] - 15, 40, 30], 2)
            elif shape_type == 2:  
                pygame.draw.rect(screen, color, [position[0] - 15, position[1] - 15, 30, 30], 2)
            elif shape_type == 3:  
                pygame.draw.polygon(screen, color, [
                    (position[0] - 15, position[1] + 15),
                    (position[0] + 15, position[1] + 15),
                    (position[0] - 15, position[1] - 15)
                ], 2)
            elif shape_type == 4: 
                height = 15 * math.sqrt(3)
                pygame.draw.polygon(screen, color, [
                    (position[0] - 15, position[1] + height/2),
                    (position[0] + 15, position[1] + height/2),
                    (position[0], position[1] - height/2)
                ], 2)
            elif shape_type == 5:  
                pygame.draw.polygon(screen, color, [
                    (position[0] - 15, position[1]),
                    (position[0], position[1] + 15),
                    (position[0] + 15, position[1]),
                    (position[0], position[1] - 15)
                ], 2)

def draw_active_shape(pos, color, shape_type):
    if color == (255, 255, 255): 
        pygame.draw.rect(screen, color, [pos[0] - 15, pos[1] - 15, 30, 30])
    else:
        if shape_type == 0:  
            pygame.draw.circle(screen, color, pos, 15, 2)
        elif shape_type == 1: 
            pygame.draw.rect(screen, color, [pos[0] - 20, pos[1] - 15, 40, 30], 2)
        elif shape_type == 2: 
            pygame.draw.rect(screen, color, [pos[0] - 15, pos[1] - 15, 30, 30], 2)
        elif shape_type == 3: 
            pygame.draw.polygon(screen, color, [
                (pos[0] - 15, pos[1] + 15),
                (pos[0] + 15, pos[1] + 15),
                (pos[0] - 15, pos[1] - 15)
            ], 2)
        elif shape_type == 4:  
            height = 15 * math.sqrt(3)
            pygame.draw.polygon(screen, color, [
                (pos[0] - 15, pos[1] + height/2),
                (pos[0] + 15, pos[1] + height/2),
                (pos[0], pos[1] - height/2)
            ], 2)
        elif shape_type == 5:  
            pygame.draw.polygon(screen, color, [
                (pos[0] - 15, pos[1]),
                (pos[0], pos[1] + 15),
                (pos[0] + 15, pos[1]),
                (pos[0], pos[1] - 15)
            ], 2)

running = True
while running:
    timer.tick(fps)
    
    screen.fill("white")
    
    mouse_pos = pygame.mouse.get_pos()
    mouse_clicked = pygame.mouse.get_pressed()[0]
    
    brush_buttons, color_buttons, color_rgbs = create_menu(active_color)
   
    if mouse_clicked and mouse_pos[1] > 70:  
        drawings.append((active_color, mouse_pos, active_shape))
    
    draw_all_shapes(drawings)
    
    if mouse_pos[1] > 70:  
        draw_active_shape(mouse_pos, active_color, active_shape)
    
   
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False  
        
        if event.type == pygame.MOUSEBUTTONDOWN: 
           
            for i in range(len(color_buttons)):
                if color_buttons[i].collidepoint(event.pos):
                    active_color = color_rgbs[i]  
            
            for button in brush_buttons:
                if button[0].collidepoint(event.pos):
                    active_shape = button[1] 
    
    pygame.display.flip()

pygame.quit()